# Financial fraud detection Transformer model

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE
from bayes_opt import BayesianOptimization
from tqdm import tqdm

# Data Preprocessing & Feature Engineering

In [12]:
# Load dataset
df = pd.read_csv('fraudTest.csv')

# Fill missing values
df.fillna(method='ffill', inplace=True)

# Outlier detection and replacement
df['amt'] = np.where(df['amt'] > df['amt'].quantile(0.95), df['amt'].quantile(0.95), df['amt'])
df['amt'] = np.where(df['amt'] < df['amt'].quantile(0.05), df['amt'].quantile(0.05), df['amt'])

# Drop non-essential columns
columns_to_drop = ['Unnamed: 0', 'cc_num', 'first', 'last', 'street', 'trans_num', 'unix_time']
df.drop(columns=columns_to_drop, axis=1, inplace=True)

# Convert dates and extract components
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['dob'] = pd.to_datetime(df['dob'])
df['age'] = df['trans_date_trans_time'].dt.year - df['dob'].dt.year
df['hour'] = df['trans_date_trans_time'].dt.hour
df['day_of_week'] = df['trans_date_trans_time'].dt.dayofweek
df.drop(columns=['trans_date_trans_time', 'dob'], axis=1, inplace=True)

# Scale numerical features
scaler = StandardScaler()
numerical_columns = ['amt', 'age', 'hour', 'day_of_week']
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Define and encode categorical columns
categorical_columns = ['gender', 'category', 'state', 'merchant', 'city', 'zip', 'job']
label_encoders = {col: LabelEncoder().fit(df[col]) for col in categorical_columns}
for col, encoder in label_encoders.items():
    df[col] = encoder.transform(df[col])

# Separate features and target
X = df.drop('is_fraud', axis=1)
y = df['is_fraud']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Applying SMOTE to balance the dataset
smote = SMOTE(sampling_strategy=0.2, random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Convert data to tensors
X_train_categorical = torch.tensor(X_train[categorical_columns].values, dtype=torch.long)
X_train_numerical = torch.tensor(X_train[numerical_columns].values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)

X_test_categorical = torch.tensor(X_test[categorical_columns].values, dtype=torch.long)
X_test_numerical = torch.tensor(X_test[numerical_columns].values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# Dataset and DataLoader
train_dataset = TensorDataset(X_train_categorical, X_train_numerical, y_train)
test_dataset = TensorDataset(X_test_categorical, X_test_numerical, y_test)

# Embedding dimensions
embedding_sizes = [(len(label_encoders[col].classes_), min(10, (len(label_encoders[col].classes_)+1)//2)) for col in categorical_columns]
num_numerical_features = X_train_numerical.shape[1]

C:\Users\shin7\AppData\Local\Temp\ipykernel_11408\4271020941.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


## Create TransformerModel

In [13]:
class TransformerModel(nn.Module):
    def __init__(self, num_numerical, embedding_sizes, num_classes=1, nhead=4, num_layers=1, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(num_classes, size) for num_classes, size in embedding_sizes])
        total_emb_size = sum(size for _, size in embedding_sizes)
        d_model = total_emb_size + num_numerical
        
        if d_model % nhead != 0:
            while d_model % nhead != 0:
                nhead -= 1
            print(f"Adjusted 'nhead' to {nhead} to be divisible by 'd_model' {d_model}")
        
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=nhead, 
            dim_feedforward=512,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.linear = nn.Linear(d_model, num_classes)
        self.activation = nn.Sigmoid()
    
    def forward(self, x_cat, x_num):
        embeddings = [embedding(x_cat[:, i]) for i, embedding in enumerate(self.embeddings)]
        x_emb = torch.cat(embeddings, dim=1)
        x_combined = torch.cat((x_emb, x_num), dim=1).unsqueeze(1)
        x_encoded = self.transformer_encoder(x_combined)
        x_out = self.linear(x_encoded.squeeze(1))
        return self.activation(x_out)

## Hyperparameter tuning

In [ ]:
def train_evaluate(learning_rate, dropout, num_layers, batch_size):
    model = TransformerModel(
        num_numerical=num_numerical_features,
        embedding_sizes=embedding_sizes,
        nhead=4,
        num_layers=int(num_layers),
        dropout=dropout
    )
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    criterion = nn.BCELoss(weight=torch.tensor([15.0]))  # Increased weight for positive class
    
    train_loader = DataLoader(train_dataset, batch_size=int(batch_size), shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=int(batch_size), shuffle=False)
    
    best_score = 0
    best_model_state = None
    patience = 3
    counter = 0
    
    for epoch in range(8):  # Increased maximum epochs
        model.train()
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} Training")
        for X_cat_batch, X_num_batch, y_batch in train_pbar:
            optimizer.zero_grad()
            outputs = model(X_cat_batch, X_num_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_pbar.set_postfix({'Loss': f'{loss.item():.4f}'})
        
        model.eval()
        y_true = []
        y_pred = []
        test_pbar = tqdm(test_loader, desc=f"Epoch {epoch+1} Evaluation")
        with torch.no_grad():
            for X_cat, X_num, y in test_pbar:
                outputs = model(X_cat, X_num)
                predictions = (outputs > 0.5).float()
                y_true.extend(y.tolist())
                y_pred.extend(predictions.flatten().tolist())
        
        f1 = f1_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        weighted_score = 0.3 * f1 + 0.7 * recall  # Weighted average
        print(f"Epoch {epoch+1}, F1: {f1:.4f}, Recall: {recall:.4f}, Weighted Score: {weighted_score:.4f}")
        
        if weighted_score == 0:
            print(f"Weighted Score is 0. Stopping early and moving to next parameter combination.")
            return 0  # Return 0 to indicate this parameter combination is not good
        
        if weighted_score > best_score:
            best_score = weighted_score
            best_model_state = model.state_dict()
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    # Load the best model state
    model.load_state_dict(best_model_state)
    return best_score

In [14]:
# Bayesian Optimization
pbounds = {
    'learning_rate': (1e-5, 1e-3),
    'dropout': (0.1, 0.5),
    'num_layers': (1, 3),
    'batch_size': (32, 256)
}

optimizer = BayesianOptimization(
    f=train_evaluate,
    pbounds=pbounds,
    random_state=42,
)

print("Starting Bayesian Optimization...")
optimizer.maximize(
    init_points=3,
    n_iter=6,
)

print("Best parameters:", optimizer.max['params'])
print("Best score (average of F1 and Recall):", optimizer.max['target'])

c:\Users\shin7\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Starting Bayesian Optimization...
|   iter    |  target   | batch_... |  dropout  | learni... | num_la... |
-------------------------------------------------------------------------
Adjusted 'nhead' to 2 to be divisible by 'd_model' 62


Epoch 1 Evaluation: 100%|██████████| 967/967 [00:02<00:00, 334.87it/s]


Epoch 1, F1: 0.2062, Recall: 0.7343, Weighted Score: 0.5758


Epoch 2 Evaluation: 100%|██████████| 967/967 [00:02<00:00, 349.74it/s]


...

Epoch 5, F1: 0.0978, Recall: 0.7016, Weighted Score: 0.5205


Epoch 6 Evaluation: 100%|██████████| 984/984 [00:03<00:00, 323.12it/s]


Epoch 6, F1: 0.1012, Recall: 0.6876, Weighted Score: 0.5117


Epoch 7 Evaluation: 100%|██████████| 984/984 [00:02<00:00, 337.99it/s]


Epoch 7, F1: 0.1022, Recall: 0.6853, Weighted Score: 0.5104
Early stopping at epoch 7
| 9         | 0.5274    | 113.8     | 0.5       | 1e-05     | 1.0       |
Best parameters: {'batch_size': 113.68655735125606, 'dropout': 0.5, 'learning_rate': 0.0007709479958358504, 'num_layers': 3.0}
Best score (average of F1 and Recall): 0.7180055716641082


## Final Model Training

In [15]:
# Train the final model with the best parameters
best_params = optimizer.max['params']
best_params['num_layers'] = int(best_params['num_layers'])
best_params['batch_size'] = int(best_params['batch_size'])

final_model = TransformerModel(
    num_numerical=num_numerical_features,
    embedding_sizes=embedding_sizes,
    nhead=4,
    num_layers=best_params['num_layers'],
    dropout=best_params['dropout']
)

optimizer = AdamW(final_model.parameters(), lr=best_params['learning_rate'])
criterion = nn.BCELoss(weight=torch.tensor([15.0]))

train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'], shuffle=False)

print("Training final model with best parameters:")
for epoch in range(10):  # You can adjust the number of epochs
    final_model.train()
    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} Training")
    for X_cat_batch, X_num_batch, y_batch in train_pbar:
        optimizer.zero_grad()
        outputs = final_model(X_cat_batch, X_num_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_pbar.set_postfix({'Loss': f'{loss.item():.4f}'})


c:\Users\shin7\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Adjusted 'nhead' to 2 to be divisible by 'd_model' 62
Training final model with best parameters:


Epoch 10 Training: 100%|██████████| 4703/4703 [01:20<00:00, 58.07it/s, Loss=0.2941]


## Final Evaluation

In [16]:
# Evaluate the final model
final_model.eval()
y_true = []
y_pred = []
test_pbar = tqdm(test_loader, desc="Final Evaluation")
with torch.no_grad():
    for X_cat, X_num, y in test_pbar:
        outputs = final_model(X_cat, X_num)
        predictions = (outputs > 0.5).float()
        y_true.extend(y.tolist())
        y_pred.extend(predictions.flatten().tolist())

f1 = f1_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print(f"\nFinal Model Results:")
print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Average Score: {(f1 + recall) / 2:.4f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

Final Evaluation:   0%|          | 0/984 [00:00<?, ?it/s]

Final Evaluation: 100%|██████████| 984/984 [00:04<00:00, 245.93it/s]



Final Model Results:
F1 Score: 0.5554
Recall: 0.7599
Average Score: 0.6576

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    110715
         1.0       0.44      0.76      0.56       429

    accuracy                           1.00    111144
   macro avg       0.72      0.88      0.78    111144
weighted avg       1.00      1.00      1.00    111144


Confusion Matrix:
[[110296    419]
 [   103    326]]
